## Create CNN Model and Optimize it using Keras Tuner

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras_tuner import HyperParameters as hp
from keras_tuner.en

In [2]:
#from keras.layers import Conv2D

In [3]:
print(tf.__version__)

2.13.0


In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [6]:
test_labels

array([9, 2, 1, ..., 8, 1, 5], dtype=uint8)

In [7]:
#scale down images so our model trains quickly
train_images=train_images/255.0
test_images=test_images/255.0

In [8]:
train_images[0].shape

(28, 28)

In [9]:
train_images.shape

(60000, 28, 28)

In [10]:
print(len(train_images))

60000


In [11]:
test_images.shape

(10000, 28, 28)

In [12]:
#Reshape the images for cnn
train_images=train_images.reshape(len(train_images),28,28,1)#len of train images is 60,000 and 1 basically represents single channel becasue this dataset is grey scale image and not RGB
test_images=test_images.reshape(len(test_images),28,28,1)

In [13]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),#This line defines the number of filters (also known as kernels) for the first convolutional layer as a hyperparameter. The hp.Int method is used to define an integer hyperparameter named 'conv_1_filter', which will be searched in the range from 32 to 128 (inclusive) with a step size of 16.
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),#kernel size or filter size
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',#we used sparse because it is efficient no need to do one hot compared to categorical cross entropy but sparse catergorical should be in int already and not other than int
              metrics=['accuracy']) 

  return model

In [14]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters #both are same (ie) from keras_tuner import Hyperparanters as hp except keras_tuner.engine can be used only within tuning function 

In [15]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=2,directory='output',project_name="Mnist Fashion")

INFO:tensorflow:Reloading Tuner from output\Mnist Fashion\tuner0.json


In [16]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 3 Complete [00h 05m 24s]
val_accuracy: 0.8646666407585144

Best val_accuracy So Far: 0.8646666407585144
Total elapsed time: 00h 05m 24s
INFO:tensorflow:Oracle triggered exit


In [17]:
model=tuner_search.get_best_models(num_models=1)[0] #The num_models=1 argument specifies that you want to retrieve only one best model.
#[0]: The [0] at the end is used to access the first (or zeroth, as indexing in many programming languages starts from 0) element of the list of best models returned by get_best_models. Since i specified num_models=1, there is only one model in the list, and [0] retrieves it.

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 112)       2912      
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        64576     
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 96)                2973792   
                                                                 
 dense_1 (Dense)             (None, 10)                970       
                                                                 
Total params: 3042250 (11.61 MB)
Trainable params: 3042250 (11.61 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 107s 63ms/step - loss: 0.3295 - accuracy: 0.8788 - val_loss: 0.3921 - val_accuracy: 0.8695
Epoch 5/10
1688/1688 [==============================] - 106s 63ms/step - loss: 0.3162 - accuracy: 0.8838 - val_loss: 0.3727 - val_accuracy: 0.8743
Epoch 6/10
1688/1688 [==============================] - 106s 63ms/step - loss: 0.2917 - accuracy: 0.8909 - val_loss: 0.4124 - val_accuracy: 0.8712
Epoch 7/10
1688/1688 [==============================] - 106s 63ms/step - loss: 0.2798 - accuracy: 0.8945 - val_loss: 0.4120 - val_accuracy: 0.8618
Epoch 8/10
1688/1688 [==============================] - 102s 61ms/step - loss: 0.2731 - accuracy: 0.8999 - val_loss: 0.4249 - val_accuracy: 0.8637
Epoch 9/10
1688/1688 [==============================] - 106s 63ms/step - loss: 0.2597 - accuracy: 0.9052 - val_loss: 0.4492 - val_accuracy: 0.8657
Epoch 10/10
1688/1688 [==============================] - 108s 64ms/step - loss: 0.2536 - accuracy: 0.9075 - val_loss: 